# A Guided Tour of Ray Core: Remote Stateful Classes

© 2019-2022, Anyscale. All Rights Reserved

📖 [Back to Table of Contents](./ex_00_tutorial_overview.ipynb)<br>
➡ [Next notebook](./ex_04_remote_classes_revisited.ipynb) <br>
⬅️ [Previous notebook](./ex_02_remote_objs.ipynb) <br>

### Overview

Actors extend the Ray API from functions (tasks) to classes. An actor is essentially a stateful worker (or a service). When a new actor is instantiated, a new worker is created or an exsisting worker is used. The methods of the actor are scheduled on that specific worker and can access and mutate the state of that worker. Like tasks, actors support CPU, GPU, and custom resource requirements.

### Learning objectives
In this this tutorial, we'll discuss Ray Actors and learn about:
 * How Ray Actors work
 * How to write a stateful Ray Actor
 * How Ray Actors can be written as a stateful distributed service

[*Remote Classes*](https://docs.ray.io/en/latest/walkthrough.html#remote-classes-actors)  (just as remote tasks) use a `@ray.remote` decorator on a Python class. 

Ray Actor pattern is powerful. They allow you to take a Python class and instantiate it as a stateful microservice that can be queried from other actors and tasks and even other Python applications. Actors can be passed as arguments to other tasks and actors. 

<img src="images/ray_worker_actor_1.png" height="40%" width="70%">

When you instantiate a remote Actor, a separate worker process is attached to a worker process and becomes an Actor process on that worker node—all for the purpose of running methods called on the actor. Other Ray tasks and actors can invoke its methods on that process, mutating its internal state if desried. Actors can also be terminated manually if needed. The examples code below show all these cases.

<img src="images/ray_worker_actor_2.png" height="40%" width="70%">

So let's look at some examples of Python classes converted into Ray Actors.

In [1]:
import logging
import time
import ray
import random
from random import randint
import numpy as np

First, let's start Ray…

In [2]:
if ray.is_initialized:
    ray.shutdown()
ray.init(logging_level=logging.ERROR)

Python version:,3.8.13
Ray version:,2.0.1
Dashboard:,http://127.0.0.1:8265


## 3. Remote class as a stateful actor pattern

To start, we'll define a class and use the decorator: `@ray.remote`

#### Example 1: Method tracking 
**Problem**: We want to keep track of who invoked a particular method. This could be a use case for telemetry data we want to track. Or what service's methods are most frequently accessed or used.

Let's use this actor to track method invocation of an actor's methods. Each instance will track who invoked it and number of times.

In [3]:
CALLERS = ["A", "B", "C"]

@ray.remote
class MethodStateCounter:
    def __init__(self):
        self.invokers = {"A": 0, "B": 0, "C": 0}
    
    def invoke(self, name):
        # pretend to do some work here
        time.sleep(0.5)
        # update times invoked
        self.invokers[name] += 1
        # return the state of that invoker
        return self.invokers[name]
        
    def get_invoker_state(self, name):
        # return the state of the named invoker
        return self.invokers[name]
    
    def get_all_invoker_state(self):
        # return the state of all invokers
        return self.invokers

In [4]:
# Create an instance of our Actor. Instances are created by
# Actor.remote(...)
worker_invoker = MethodStateCounter.remote()
worker_invoker

Actor(MethodStateCounter, 30af8a97f4bb2b5b852789f501000000)

Iterate and call the `invoke()` method by random callers and keep track of who called it.

In [5]:
for _ in range(10):
    name = random.choice(CALLERS)
    worker_invoker.invoke.remote(name)

Invoke a random caller and fetch the value or invocations of a random caller

In [6]:
for _ in range(5): 
    random_name_invoker = random.choice(CALLERS)
    # Use comprehension to invoke each random invoker
    times_invoked = ray.get(worker_invoker.invoke.remote(random_name_invoker))
    print(f"Named caller: {random_name_invoker} called {times_invoked}")

Named caller: B called 5
Named caller: B called 6
Named caller: A called 3
Named caller: A called 4
Named caller: B called 7


Fetch the count of all callers

In [7]:
print(ray.get(worker_invoker.get_all_invoker_state.remote()))

{'A': 4, 'B': 7, 'C': 4}


**Note** that we did not have to reason about where and how the actors are scheduled.

We did not worry about the socket connection or IP addresses where these actors reside. All that's abstracted away from us. All that's handled by Ray.

All we did is write Python code, using Ray core APIs, convert our classes into distributed stateful services!

### Any questions??

#### Example 2: Parameter Server distributed application with Ray Actors 

**Problem**: We want to update weights and gradients, computed by workers, at a central server. Parameter server is one way to to have distributed training workers update and synchronize their weights. This is a simple illustration of how you may approach a use case where you what some sort of aggregation from each distributed worker.


Let's use Python class and convert that to a remote Actor class actor as a Parameter Server. 

This is a common example in machine learning where you have a central Parameter server updating gradients from other worker processes computing individual gradients. 

<img src="https://terrytangyuan.github.io/img/inblog/mpi-operator-1.png" width="60%" height="30%">

In [8]:
@ray.remote
class ParameterSever:
    def __init__(self):
        # Initialized our gradients to zero
        self.params = np.zeros(10)

    def get_params(self):
        # Return current gradients
        return self.params

    def update_params(self, grad):
        # Update the gradients 
        self.params -= grad

Define a worker or task as a function for a remote Worker. This could be a machine learning  function that computes gradients and sends them to the parameter server.

It takes an instance of the partameter server has an argument.

In [9]:
@ray.remote
def worker(ps):         # It takes an actor handle or instance as an argument
    # Iterate over some epoch
    for i in range(25):
        time.sleep(1.5)  # this could be your loss function computing gradients
        grad = np.ones(10)
        # update the gradients in the parameter server
        ps.update_params.remote(grad)

Start our Parameter Server actor. This will be scheduled as a worker process on a remote Ray node. You invoke its `ActorClass.remote(...)` to instantiate an Actor instance of that type.

In [10]:
param_server = ParameterSever.remote()
param_server

Actor(ParameterSever, 3e301efc21584adf211eda3201000000)

Let's get the initial values of the parameter server

In [11]:
print(f"Initial params: {ray.get(param_server.get_params.remote())}")

Initial params: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


### Create Workers remote tasks computing gradients
Let's create three separate worker tasks as our machine learning tasks that compute gradients. These will be scheduled as tasks on a Ray cluster.

You can use list comprehension. Quite Pythonic!

If we need more workers to scale, we can always bump them up.

**Note**: We are sending the `parameter_server` as an argument to the remote
worker task.

In [12]:
[worker.remote(param_server) for _ in range(3)]

[ObjectRef(1e360ffa862f8fe3ffffffffffffffffffffffff0100000001000000),
 ObjectRef(18b2ad3c688fb947ffffffffffffffffffffffff0100000001000000),
 ObjectRef(dc746dc61b2c1923ffffffffffffffffffffffff0100000001000000)]

Now, let's iterate over a loop and query the Parameter Server 
as the workers are running independently and updating the gradients

In [13]:
for _i in range(20):
    print(f"Updated params: {ray.get(param_server.get_params.remote())}")
    time.sleep(1)

Updated params: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Updated params: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Updated params: [-3. -3. -3. -3. -3. -3. -3. -3. -3. -3.]
Updated params: [-3. -3. -3. -3. -3. -3. -3. -3. -3. -3.]
Updated params: [-6. -6. -6. -6. -6. -6. -6. -6. -6. -6.]
Updated params: [-9. -9. -9. -9. -9. -9. -9. -9. -9. -9.]
Updated params: [-11. -11. -11. -11. -11. -11. -11. -11. -11. -11.]
Updated params: [-12. -12. -12. -12. -12. -12. -12. -12. -12. -12.]
Updated params: [-15. -15. -15. -15. -15. -15. -15. -15. -15. -15.]
Updated params: [-18. -18. -18. -18. -18. -18. -18. -18. -18. -18.]
Updated params: [-18. -18. -18. -18. -18. -18. -18. -18. -18. -18.]
Updated params: [-21. -21. -21. -21. -21. -21. -21. -21. -21. -21.]
Updated params: [-24. -24. -24. -24. -24. -24. -24. -24. -24. -24.]
Updated params: [-24. -24. -24. -24. -24. -24. -24. -24. -24. -24.]
Updated params: [-27. -27. -27. -27. -27. -27. -27. -27. -27. -27.]
Updated params: [-30. -30. -30. -30. -30. -30. -30. -30. -30

### Look at the Ray Dashboard

You should see Actors running as process on the workers nodes
 * Parameter Server
 
Also, click on the `Actors` to view more metrics and data on individual Ray Actors

Finally, shutdown Ray

In [14]:
ray.shutdown()

### Any questions?

### Exercises

1. Modify the Actor class `MethodStateCounter` and add/modify methods that return the following:
 * Get number of times an invoker `name` was called
 * Get a list of values computed by invoker `name` 
 * Get state of all invokers
 
2. Modify method `invoke` to return a random int value between [5, 25]

### Next Step

We covered how to use Ray Actors and write a distributed service. Next, let's explore
how Actors can be used to write more distributed applications using Ray Actor Tree pattern.

Let's move on to the [Ray Actor Revised](ex_04_remote_classes_revisited.ipynb).

## Homework

Read these references as cure to your insomnia :-)

 * [Writing your First Distributed Python Application with Ray](https://www.anyscale.com/blog/writing-your-first-distributed-python-application-with-ray)
 * [Using and Programming with Actors](https://docs.ray.io/en/latest/actors.html)

📖 [Back to Table of Contents](./ex_00_tutorial_overview.ipynb)<br>
➡ [Next notebook](./ex_04_remote_classes_revisited.ipynb) <br>
⬅️ [Previous notebook](./ex_02_remote_objs.ipynb) <br>